In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation


nn = Sequential()
nn.add(Dense(10, activation="relu", input_shape=(10,)))
nn.add(Dense(5))

W0625 11:41:10.341013 140735499895680 deprecation_wrapper.py:119] From /anaconda3/envs/TDS/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0625 11:41:10.356137 140735499895680 deprecation_wrapper.py:119] From /anaconda3/envs/TDS/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0625 11:41:10.360365 140735499895680 deprecation_wrapper.py:119] From /anaconda3/envs/TDS/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



$$softmx(z)= P(c_j|z_i) = \frac{e^{z_i}}{\sum{e^{z_i}}}$$
<p>softmax models a probability of z_i being in a class c_j as a multinomial distribution. For only 1 class. The softmax is an approximation
to argmax. There are some problems with softmax as an approximator to argmax. softmax is not scale invariant, smaller
values yield different probabilities, softmax[.1,.2.3,.4] is different than softmax[1,2,3,4]. The effect is still
the same we want the one which is the most probable to have the highest value and the resulting probabilites should
sum to 1. </p>
<p>sigmoid models z as a bernoulli distribution. Use a binary CE loss w/sigmoid for a multiclass multilabel. We want [0,.5,.5] as an output to represent the z_i is a member of classes 2 and 3. </p>
$$sigmoid(z) = \frac{1}{1+e^{-z_i}}$$
<p>Cross entropy</p>

In [11]:
import numpy as np

def softmax(z):
    return np.exp(z)/np.sum(np.exp(z))
#4 is highest value. Even if we normalize it is still the largest value
print(softmax([1.0, 2.0, 3.0, 4.0]))
def sigmoid(z):
    return [1/(1+np.exp(-x)) for x in z]
#sigmoid prob does not add up to 1! Use for multilabel multiclass if paired w/BCE and multiple outputs
print(sigmoid([1,2,3,4]))


[0.0320586  0.08714432 0.23688282 0.64391426]
[0.7310585786300049, 0.8807970779778823, 0.9525741268224334, 0.9820137900379085]
